In [1]:
import time
import numpy as np
import pandas as pd

from sklearn.utils import shuffle
from tensorflow.keras.backend import backend
from tensorflow.keras.models import *
from tensorflow.keras.layers import * 

import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)
#print(gpus)

print ("============================================================")
print (" CNN for next event prediction                              ")
print ("============================================================")


start_time = time.time()
acc_list = []

df = pd.read_excel("table.xlsx")
df.dropna(inplace = True) 

n = 10
for ii in range(1, n+1):
    
    time_02 = time.time()

    df = shuffle(df) 

    size = df.shape[0]
    t = int(size * 0.8)         # train part size

    x = df.values[:size, 0:7]
    y = np.empty(size, dtype=str)

    for i in range(x.shape[0]):
        y[i] = x[i, 0][-1]
        x[i, 0] = x[i, 0][:-1]

    X = np.empty((x.shape[0], x.shape[1]), dtype='int32')

    for d in range(x.shape[1]):        
        xs = list(set(x[:, d]))

        xd = {xs[i]: i for i in range(len(xs))}
        for l in range(x.shape[0]):
            X[l][d] = xd[x[l][d]]


    XF_train = X[0:t, 1:]           # train features 
    XF_test  = X[t:size, 1:]        # test features

    yn = np.empty((size), dtype='int32')

    ys = list(set(y[:]))
    yd = {ys[i]: i for i in range(len(ys))}
    for l in range(len(yn)):
        yn[l] = yd[y[l]]

    y_train = yn[0:t]
    y_test  = yn[t:size]

    # all unique characters to the set
    events = set()
    for seq in x[:, 0]:
        for event in seq:
            events.add(event)

    events = list(events)
    event_to_id = {t:i+1 for i,t in enumerate(events)}

    max_seq_len = 8
    seq_events_numbered = np.zeros((x.shape[0], max_seq_len), dtype='int32')

    for i in range(seq_events_numbered.shape[0]):
        for k in range(len(x[i][0])):
            seq_events_numbered[i][k] = event_to_id[x[i][0][k]]

    S_train = seq_events_numbered[0:t, :]       # train sequences
    S_test  = seq_events_numbered[t:size, :]    # test  sequences

    
    def define_model(seq_length, vocab_size, features_n): # for sequences: length=8, vocab_size=9 (include ' ') 

        input_seq = Input(shape=(seq_length,)) 
        
        # sequences
        seq = Embedding(vocab_size, 100)(input_seq)
        seq = Conv1D(128, kernel_size=8, activation='relu')(seq)
        seq = Dropout(0.05)(seq)
        seq = Flatten()(seq)
        
        # features
        input_features = Input(shape=(features_n, ))
        features = Dense(128, activation='relu')(input_features)
        features = Dropout(0.05)(features)
        features = Flatten()(features)

        # merged sequences and features
        merged = concatenate([seq, features]) # merged sequences and features
        merged = Dense(128, activation='relu')(merged)
        merged = Dropout(0.05)(merged)
        merged = Dense(32, activation='relu')(merged)
        
        outputs = Dense(8, activation='softmax')(merged)
        model = Model(inputs=[input_seq, input_features], outputs=outputs)

        model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', 
                      metrics=['sparse_categorical_accuracy'])

        return model


    # define model
    model = define_model(8, 9, 6)

    # fit model
    model.fit([S_train, XF_train], y_train, epochs=70, batch_size=100, verbose=0)

    loss, acc = model.evaluate([S_test, XF_test], y_test, verbose=0)
    acc_list.append(acc)
    
    print("Training & evaluation", ii, ": accuracy = %0.3f" % acc, "time = %0.2f" % (time.time() - time_02))

time_03 = time.time()

print ("\nModel training & evaluation time (mean): %0.2f" % ((time_03 - start_time)/n))   
print ("Accuracy (mean): %.3f" % np.mean((acc_list)), ", Variance: %.6f" % (np.var(acc_list)), 
       ", Standard deviation: %.6f" % (np.std(acc_list)))
print ("==========================================================================")


 CNN for next event prediction                 
Training & evaluation 1 : accuracy = 0.942 time = 9.57
Training & evaluation 2 : accuracy = 0.921 time = 7.93
Training & evaluation 3 : accuracy = 0.929 time = 7.91
Training & evaluation 4 : accuracy = 0.931 time = 7.97
Training & evaluation 5 : accuracy = 0.923 time = 8.04
Training & evaluation 6 : accuracy = 0.926 time = 7.93
Training & evaluation 7 : accuracy = 0.931 time = 7.93
Training & evaluation 8 : accuracy = 0.934 time = 7.95
Training & evaluation 9 : accuracy = 0.940 time = 7.97
Training & evaluation 10 : accuracy = 0.930 time = 8.17

Model training & evaluation time (mean): 8.17
Accuracy (mean): 0.931 , Variance: 0.000041 , Standard deviation: 0.006442
